In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv1D,MaxPool1D

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import classification_report
from joblib import dump, load
import queue
import pandas as pd
import numpy as np
import glob, os, sys
pd.options.mode.chained_assignment = None

c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
with open('./Dataset/csv/Attack_merge.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
    
if not os.path.exists("./models"):
    os.mkdir("./models")

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/DT.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

In [2]:
model = RandomForestClassifier(n_estimators=20, max_depth=4)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/RF.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

NameError: name 'X_train' is not defined

In [4]:
model = LogisticRegression(C=3)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/LR.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

0.9075788061703555
              precision    recall  f1-score   support

           0       0.91      0.91      0.91     22799
           1       0.90      0.91      0.91     21931

    accuracy                           0.91     44730
   macro avg       0.91      0.91      0.91     44730
weighted avg       0.91      0.91      0.91     44730



In [5]:
model = XGBClassifier(booster="gbtree", min_split_loss=7.100747859845302e-07, alpha=0.0010277375135306342, 
                        max_depth=9, eta=0.6737504946980999, gamma=6.101266632438708e-07, grow_policy="lossguide")
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/XGB.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:04:10] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9765034652358596
              precision    recall  f1-score   support

           0       0.98      0.97      0.98     22799
           1       0.97      0.98      0.98     21931

    accuracy                           0.98     44730
   macro avg       0.98      0.98      0.98     44730
weighted avg       0.98      0.98      0.98     44730



c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [6]:
model = LinearSVC(C = 2, verbose=False)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/SVM.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

0.9097250167672702
              precision    recall  f1-score   support

           0       0.91      0.91      0.91     22799
           1       0.91      0.91      0.91     21931

    accuracy                           0.91     44730
   macro avg       0.91      0.91      0.91     44730
weighted avg       0.91      0.91      0.91     44730



c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [6]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/KNN.joblib')
prediction = model.predict(X_test)
print(prediction)
print(classification_report(y_test,prediction))

0.9753856472166331
[0 0 0 ... 1 1 0]
              precision    recall  f1-score   support

           0       0.98      0.97      0.98     22652
           1       0.97      0.98      0.98     22078

    accuracy                           0.98     44730
   macro avg       0.98      0.98      0.98     44730
weighted avg       0.98      0.98      0.98     44730



In [63]:
batch_size = 32
X_train = np.array(X_train)
y_train = np.array(y_train)

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./models/DNN4_Best_Model.h5', save_weights_only=False, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, epsilon=1e-4, mode='min')

dnn4 = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.01),
    tf.keras.layers.Dense(2, activation='softmax')
])

dnn4.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']);

dnn4.fit(X_train, y_train, batch_size=batch_size, epochs=10, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)
model = tf.keras.models.load_model('./models/DNN4.h5')

Epoch 1/10
4194/4194 [==============================] - 28s 6ms/step - loss: 0.2553 - accuracy: 0.8863 - val_loss: 0.1968 - val_accuracy: 0.9101
Epoch 2/10
4194/4194 [==============================] - 26s 6ms/step - loss: 0.2032 - accuracy: 0.9076 - val_loss: 0.1767 - val_accuracy: 0.9211
Epoch 3/10
4194/4194 [==============================] - 28s 7ms/step - loss: 0.1994 - accuracy: 0.9084 - val_loss: 0.1758 - val_accuracy: 0.9214
Epoch 4/10
4194/4194 [==============================] - 27s 6ms/step - loss: 0.1983 - accuracy: 0.9095 - val_loss: 0.1848 - val_accuracy: 0.9181
Epoch 5/10
4194/4194 [==============================] - 28s 7ms/step - loss: 0.1938 - accuracy: 0.9118 - val_loss: 0.1797 - val_accuracy: 0.9189

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 6/10
4194/4194 [==============================] - 28s 7ms/step - loss: 0.1856 - accuracy: 0.9159 - val_loss: 0.1731 - val_accuracy: 0.9215
Epoch 7/10
4194/4194 [==========================

In [3]:
model.save("./models/DNN4.h5")
model = tf.keras.models.load_model('./models/DNN4.h5')

NameError: name 'model' is not defined

In [71]:
preds = model.predict(X_test)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels[0:,1])
f1 = sklearn.metrics.f1_score(y_test, pred_labels[0:,1], average="binary")
prec = sklearn.metrics.precision_score(y_test, pred_labels[0:,1], average="binary")
recall = sklearn.metrics.recall_score(y_test, pred_labels[0:,1], average="binary")
print("accuracy: " + str(accuracy))
print("f1: " + str(f1))
print("prec: " + str(prec))
print("recall: " + str(recall))

accuracy: 0.9208361278783814
f1: 0.9181404165799755
prec: 0.9277271665498715
recall: 0.9087497711879919


In [3]:
y_train_ex = tf.expand_dims(y_train, axis=1)
X_train_ex = tf.expand_dims(X_train, axis=2)
y_test_ex = tf.expand_dims(y_test, axis=1)
X_test_ex = tf.expand_dims(X_test, axis=2)
y_train_np = np.array(y_train_ex)
X_train_np = np.array(X_train_ex)
y_test_np = np.array(y_test_ex)
X_test_np = np.array(X_test_ex)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding, GlobalAveragePooling1D
epochs = 100
batch_size = 256

model = Sequential()
model.add(Conv1D(filters=100, kernel_size=5, padding='same',activation='relu',strides=1, input_shape=(X_train_ex.shape[1],1))) 
model.add(Dropout(0.2))
model.add(Conv1D(filters=200, kernel_size=5, padding='same', activation='relu',strides=1))
model.add(Dropout(0.2))
model.add(Conv1D(filters=400, kernel_size=10, padding='same', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

model.fit(X_train_ex, y_train_np, epochs=epochs, batch_size=batch_size,) 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 10, 100)           600       
_________________________________________________________________
dropout_10 (Dropout)         (None, 10, 100)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 10, 200)           100200    
_________________________________________________________________
dropout_11 (Dropout)         (None, 10, 200)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 10, 400)           800400    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 400)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2000)             

In [4]:
#model.save("./models/CNN.h5")
model = tf.keras.models.load_model("./models/CNN.h5")

In [5]:
preds = model.predict(X_test_ex)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_test_ex, pred_labels)
f1 = sklearn.metrics.f1_score(y_test_ex, pred_labels, average="binary")
prec = sklearn.metrics.precision_score(y_test_ex, pred_labels, average="binary")
recall = sklearn.metrics.recall_score(y_test_ex, pred_labels, average="binary")
print("accuracy: " + str(accuracy))
print("f1: " + str(f1))
print("prec: " + str(prec))
print("recall: " + str(recall))

accuracy: 0.9314106863402638
f1: 0.9291585850189341
prec: 0.9477154969382948
recall: 0.9113144306549507


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [6]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train_np.shape[1], X_train_np.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(1, activation='sigmoid'))
regressor.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
regressor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 10, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)               

In [10]:

# 進行訓練
regressor.fit(X_train_ex, y_train_ex, epochs = 20, batch_size = 32)

Epoch 1/20
5592/5592 [==============================] - 69s 11ms/step - loss: 0.3544 - accuracy: 0.9227
Epoch 2/20
5592/5592 [==============================] - 63s 11ms/step - loss: 0.1407 - accuracy: 0.9546
Epoch 3/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.1009 - accuracy: 0.9626
Epoch 4/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0910 - accuracy: 0.9641
Epoch 5/20
5592/5592 [==============================] - 66s 12ms/step - loss: 0.0876 - accuracy: 0.9649
Epoch 6/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0891 - accuracy: 0.9630
Epoch 7/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0860 - accuracy: 0.9647
Epoch 8/20
5592/5592 [==============================] - 63s 11ms/step - loss: 0.0828 - accuracy: 0.9654
Epoch 9/20
5592/5592 [==============================] - 64s 11ms/step - loss: 0.0835 - accuracy: 0.9648
Epoch 10/20
5592/5592 [==============================] - 63s 11m

In [14]:
regressor.save("./models/LSTM.h5")
model = tf.keras.models.load_model("./models/LSTM.h5")

In [15]:
preds = model.predict(X_test_ex)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_test_ex, pred_labels)
f1 = sklearn.metrics.f1_score(y_test_ex, pred_labels, average="binary")
prec = sklearn.metrics.precision_score(y_test_ex, pred_labels, average="binary")
recall = sklearn.metrics.recall_score(y_test_ex, pred_labels, average="binary")
print("accuracy: " + str(accuracy))
print("f1: " + str(f1))
print("prec: " + str(prec))
print("recall: " + str(recall))

accuracy: 0.9692823608316566
f1: 0.9683847215830649
prec: 0.9694554501059615
recall: 0.9673163556127609
